# 안녕하세요. AIFactory입니다 ☺️
이 노트북은 제출을 위한 가이드라인 노트북입니다.  

Colab에서 작업 하실 경우, 반드시 구글 드라이브에 사본을 저장을(파일 -> Drive에 사본 저장) 하신 후에 사용해주세요.
  
이번 태스크는 스코어 경쟁을 통해 입상자를 선별하는 대회가 아닌,  
AIF의 **추론 자동화**를 경험해보고 이 과정에서 불편한 점, 개선 되었으면 하는 점 등  
여러분들의 다양한 의견을 듣기 위해서이니 다양한 의견을 공유해 주시면 감사드리겠습니다.



# 주의 사항

### Colab 환경에서 작업 하시는 경우
- Colab 노트북 공유 권한을 ***공유 -> 링크가 있는 모든 사용자*** 로 설정해주세요.
- 구글 드라이브 마운트는 하지 말아주세요.
    - AIF 추론 환경에서는 Colab 전용 라이브러리를 사용 할 수 없으며,
    - 참가자분의 개인 구글 드라이브에 접근 할 수 없습니다.
</br></br>

### 개인 로컬 환경에서 작업 하시는 경우
- 제출 스크립트 파일 이름을 반드시 ***task.ipynb***로 해주세요.
</br></br>

### 공통
- 제출 스크립트에 학습 코드를 포함하지 말아주세요. 학습 코드는 별도 작업 공간에서 진행해주세요.
- 현재 작업중인 경로에서 모델 추론에 불필요한 파일(학습 데이터 등...)이 제출되지 않도록 주의해주세요.
- 모든 경로는 현재 작업중인 스크립트 파일을 기준으로 **상대 경로**로 설정해주세요.
- 이번 베타 테스트의 추론 환경은 CPU만 사용하여 평가를 진행하니 참고하여 코드를 작성해주세요.  
(CPU: 8 Core / REM: 12GB / Storage: 20GB)
- AIF의 추론 환경은 Colab과 완전히 동일하지 않으니, 추론 환경에 설치된 패키지 항목을 참고하신 후 필요한 패키지를 설치해주세요.
</br></br>

</br></br>
**추론 환경에 설치된 패키지 리스트**
```
tensorflow==2.11.1
keras-cv
torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1
albumentations
transformers
scikit-learn
numpy
Cython
pycocotools
pandas
jupyter
notebook
matplotlib
seaborn
plotly
Pillow
opencv-python
scikit-image
```

# 0. 필요 패키지 설치

In [ ]:
# aifactory 패키지는 반드시 설치해주세요!
!pip install -U aifactory

In [ ]:
# 필요한 패키지를 추가로 설치해주세요.
!pip install gdown
!pip install tfts
# ...

In [ ]:
# 패키지 임포트
import os
import random
import numpy as np
import torch
import tensorflow as tf
# ...

In [ ]:
# 무작위성 통제
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
# ...

# 1. 모델 로드 함수를 정의해주세요.
- 모델 로드 함수는 참가자 분들께서 이미 학습 완료한 모델 가중치 파일을 읽어와 메모리에 올려 놓은 후 반환 되어야 합니다.
- 모델 가중치 파일을 현재 노트북 파일 기준 ***동일 경로 또는 하위 경로***에 있어야하며, ***상대 경로***로 설정 되어야 합니다.
- Colab에서 작업 하시는 경우 가중치 파일을 수동으로 업로드 하시거나 또는,  
개인 구글 드라이브에 업로드 후(***공유 권한: 링크가 있는 모든 사용자***) `gdown`을 통해 다운 받아 사용하세요.
- 이 함수가 받는 인자(Arguments)는 없거나, default 값으로 설정 되어야 합니다.
- 함수 명칭은 자유롭게 정의해주세요.


```python
# Example 1
import gdown

def tf_load_model():
    weights_path = './my_weights.h5'
    gdown.downalod("GoolgeDriveSharedLink", weights_path, fuzzy=True)
    ...
    model.load_weights(weights_path)
    return model

# Example 2
def torch_load_model(weights_path='./my_weights.h5'):
    ...
    model.load_state_dict(torch.load(weights_path))
    model.eval()
    return model
```


In [ ]:
def load_model():
    # 모델 아키텍쳐
    time_series_model = Sequential([
        layers.LSTM(...)
        ...
        layers.Dense(1, )
    ])

    # 모델 가중치 로드
    time_series_model.load_weights('./my_weights.h5')

    # ...

    return time_series_model

# 2. 예측 함수를 정의해주세요.
- 예측 함수는 반드시 **2개의 인자**를 다음과 같이 순서대로 받아야합니다.
    - 첫 번째 인자 - AIF가 제공하는 평가 입력 데이터(X_test)
    - 두 번째 인자 - 가중치를 읽어온 모델

- 첫 번째 인자(X_test)는 ***time series 평가셋***의 내용이이 담긴 ***array***를 전달합니다.  
    - `X_test 는 (1728, 6, 6) shape의 array
    - X_test가 제공되는 형식을 참고하여 예측 및 전/후 처리를 통해 **출력 규칙**에 맞게 결과를 반환해주세요.
    
- **출력 규칙**
    - 전체 예측값으로로 구성된 ***array***를 반환해주세요.
    - pred 형태 예시 : array([1.1,1.2,1.3,1.4,1.5,1.6..]), shape = (1728,)



In [ ]:
# 예측 함수는 AIF에서 제공하는 X_test의 입력 정보와, 메모리에 올라간 사전 학습 모델을 입력 받습니다.
def predict(X_test, model):

    result = model.predict(test_x)
    result = result.reshape(-1) # shape : (1728,)
    
    return result

# 3. submit 함수를 정의해 주세요.  
- 이 함수의 명칭은 반드시 **submit** 으로 해주세요.
- **submit** 함수는 AIF가 참가자분들이 정의한 2개의 함수를 전달받아 실행하기 위해 사용됩니다.
- **submit** 함수를 통해 위에서정의한 함수를 **1.모델 로드 함수**와 **2.예측 함수** 순으로 반환해 주세요.
- AIF가 submit 함수를 통해 채점하는 방식은 다음과 같으니 참고해주세요.  
    ```python
    # AIF 채점 코드 예시
    X_test = array([[1.17766667e-01, 2.47846667e+02, 1.99000000e+00, 0.00000000e+00,
        6.83333333e-01, 6.83333333e-01],
       [5.95000000e-02, 2.45663500e+02, 2.62666667e+00, 0.00000000e+00,
        0.00000000e+00, 6.83333333e-01],
       [1.65900000e-01, 2.40940667e+02, 6.06666667e+00, 0.00000000e+00,
        0.00000000e+00, 6.50000000e-01],
       [1.40533333e-01, 2.41279167e+02, 9.60666667e+00, 0.00000000e+00,
        0.00000000e+00, 1.39166667e+01],
       [5.73000000e-02, 2.42149167e+02, 8.90333333e+00, 0.00000000e+00,
        0.00000000e+00, 1.41333333e+01],
       [1.28700000e-01, 2.42520000e+02, 3.43000000e+00, 0.00000000e+00,
        5.66666667e-01, 6.66666667e-01
        ...
        ]])
    y_test = array([0.3991    , 0.3269    , 2.134342    , 1.384736 ... ])

    load_model, predict = submit()
    models = load_model() # 참가팀 모델 로드 함수 실행
    y_pred = predict(X_test, models) # 참가팀 예측 함수 실행

    from sklearn.metrics import mean_absolute_error
    score = mean_absolute_error(y_test, y_pred)
    ```

In [ ]:
def submit():
    return load_model, predict

# 4. 제출하기
- aifactory 라이브러리를 통해 submit 함수를 제출합니다.
- aif.submit 함수는 반드시 `if __name__ == "__main__":` 아래에 작성해주세요.
- aifactory.score.submit 함수는 3개의 인자를 받습니다.
    - ***model_name*** - 제출시 리더보드에 기록하고자 하는 문구
    - ***key*** - 대회 참가자에게 제공되는 TASK_KEY  
    - ***func*** - submit 함수

- TASK_KEY 확인 방법
    - https://aifactory.space/competition/detail/2369 페이지 접속
    - 우측 상단 `내정보` -> `태스크 정보` 클릭
    - 3분야 : 시계열 모델 퀘스트에서 `사용자 키 보기` 클릭

In [ ]:
import aifactory.score as aif

if __name__ == "__main__":
    aif.submit(
        model_name="MyFirstSubmission",
        key='TASK_KEY',
        func=submit)